In [1]:
%load_ext autoreload
%autoreload 2
import sys
import IPython

install_needed = True
#install_needed = False

if install_needed:
    print("===> Installing deps and restarting kernel. Please change 'install_needed = False' and run this code cell again.")
    !{sys.executable} -m pip install -U "argparse" "torch==1.13.1"  "torchvision==0.14.1"  "awscli==1.27.68" "boto3==1.26.68" "botocore==1.29.68" "datasets==1.18.4" "sagemaker==2.143.0" "s3fs==0.4.2" "s3transfer==0.6.0" "transformers==4.27.4" "nvidia-cublas-cu11==11.10.3.66" "nvidia-cuda-nvrtc-cu11==11.7.99" "nvidia-cuda-runtime-cu11==11.7.99" "nvidia-cudnn-cu11==8.5.0.96"
    IPython.Application.instance().kernel.do_shutdown(True)

===> Installing deps and restarting kernel. Please change 'install_needed = False' and run this code cell again.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [1]:
!pip freeze

aiohttp==3.8.3
aiosignal==1.3.1
anyio @ file:///home/conda/feedstock_root/build_artifacts/anyio_1666191106763/work/dist
argon2-cffi @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi_1640817743617/work
argon2-cffi-bindings @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi-bindings_1649500320262/work
async-timeout==4.0.2
asynctest==0.13.0
attrs @ file:///home/conda/feedstock_root/build_artifacts/attrs_1671632566681/work
autovizwidget==0.20.3
awscli==1.27.68
Babel @ file:///home/conda/feedstock_root/build_artifacts/babel_1667688356751/work
backcall @ file:///home/conda/feedstock_root/build_artifacts/backcall_1592338393461/work
backports.functools-lru-cache @ file:///home/conda/feedstock_root/build_artifacts/backports.functools_lru_cache_1618230623929/work
bcrypt==4.0.1
beautifulsoup4 @ file:///home/conda/feedstock_root/build_artifacts/beautifulsoup4_1649463573192/work
bleach @ file:///home/conda/feedstock_root/build_artifacts/bleach_1656355450470/work
bokeh==2.

# MLOps with SageMaker Pipelines


## Prerequisites

Reference: 

https://github.com/gonsoomoon-ml/SageMaker-Pipelines-Step-By-Step
https://github.com/gonsoomoon-ml/SageMaker-Pipelines-Step-By-Step/tree/main/phase01
https://github.com/gonsoomoon-ml/SageMaker-Pipelines-Step-By-Step/tree/main/phase02

- SageMaker Pipelines SDK: https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html
- Caching Pipeline Steps: https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-caching.html
- AWS AIML Blog: Use a SageMaker Pipeline Lambda step for lightweight model deployments: https://aws.amazon.com/de/blogs/machine-learning/use-a-sagemaker-pipeline-lambda-step-for-lightweight-model-deployments/

Note:
- 본 노트북을 실행하려면 `AmazonSageMakerFullAccess`와 `AmazonSageMakerPipelinesIntegrations` policy를 추가

In [2]:
import boto3
import os
import numpy as np
import sagemaker
import sys
import time

import sagemaker
import sagemaker.huggingface
from sagemaker.huggingface import HuggingFace, HuggingFaceModel

from sagemaker.workflow.parameters import ParameterInteger, ParameterFloat, ParameterString

from sagemaker.lambda_helper import Lambda

from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.huggingface.processing import HuggingFaceProcessor

from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import CacheConfig, ProcessingStep

from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep

from sagemaker.processing import ScriptProcessor
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.step_collections import CreateModelStep, RegisterModel

from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo,ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet

from sagemaker.workflow.pipeline import Pipeline, PipelineExperimentConfig
from sagemaker.workflow.execution_variables import ExecutionVariables

In [3]:
sess = sagemaker.Session()
region = sess.boto_region_name

# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sagemaker_session.default_bucket()}")
print(f"sagemaker session region: {sagemaker_session.boto_region_name}")

sagemaker role arn: arn:aws:iam::353411055907:role/sagemaker-notebook-test1-SageMakerIamRole-3LWOV1Z5K6V0
sagemaker bucket: sagemaker-us-east-1-353411055907
sagemaker session region: us-east-1



<br>

## 1. Defining the Pipeline
---

### 1.1. Pipeline parameters

References: 
- 개발자 가이드: https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-parameters.html

In [4]:
# S3 prefix where every assets will be stored
s3_prefix = "GP-LJP-mlops"

# s3 bucket used for storing assets and artifacts
bucket = sagemaker_session.default_bucket()

# aws region used
region = sagemaker_session.boto_region_name

# base name prefix for sagemaker jobs (training, processing, inference)
base_job_prefix = s3_prefix

# Cache configuration for workflow
cache_config = CacheConfig(enable_caching=True, expire_after="7d")

# package versions
transformers_version = '4.17.0'
pytorch_version = '1.10.2'
py_version = "py38"

model_id_ = "lawcompany/KLAID_LJP_base"
tokenizer_id_ = "lawcompany/KLAID_LJP_base"
dataset_name_ = "lawcompany/KLAID"

model_id = ParameterString(name="ModelId", default_value=model_id_)
tokenizer_id = ParameterString(name="TokenizerId", default_value=tokenizer_id_)
dataset_name = ParameterString(name="DatasetName", default_value=dataset_name_)

### 1.2. Processing Step

빌트인 `SKLearnProcessor`를 통해 전처리 스텝을 정의합니다. 

References: 
- AWS AIML Blog: https://aws.amazon.com/ko/blogs/machine-learning/use-deep-learning-frameworks-natively-in-amazon-sagemaker-processing/
- 개발자 가이드: https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing

In [5]:
processing_instance_type = ParameterString(name="ProcessingInstanceType", default_value="ml.m5.xlarge")
processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)
processing_script = ParameterString(name="ProcessingScript", default_value="./src/processing_sklearn.py")

In [6]:
processing_output_destination = f"s3://{bucket}/{s3_prefix}/data"

sklearn_processor = SKLearnProcessor(
    instance_type="ml.m5.xlarge", 
    instance_count=processing_instance_count,
    framework_version="1.0-1",    
    base_job_name=base_job_prefix + "-preprocessing",
    sagemaker_session=sagemaker_session,    
    role=role
)

step_process = ProcessingStep(
    name="ProcessDataForTraining",
    #cache_config=cache_config,
    processor=sklearn_processor,
    job_arguments=["--model_id", model_id_,
                   "--tokenizer_id", tokenizer_id_,
                   "--dataset_name", dataset_name_,
                   "--transformers_version", transformers_version,
                   "--pytorch_version", pytorch_version
                  ],
    outputs=[
        ProcessingOutput(
            output_name="train",
            destination=f"{processing_output_destination}/train",
            source="/opt/ml/processing/train",
        ),
        ProcessingOutput(
            output_name="validation",
            destination=f"{processing_output_destination}/test",
            source="/opt/ml/processing/validation",
        ),
        ProcessingOutput(
            output_name="test",
            destination=f"{processing_output_destination}/test",
            source="/opt/ml/processing/test",
        )        
    ],
    code="./src/processing_sklearn.py"
)

### 1.3. Model Training Step

이전 랩에서 진행한 훈련 스크립트를 그대로 활용하여 훈련 스텝을 정의합니다. SageMaker Pipelines에 적용하기 위해 워크플로 파라메터(`ParameterInteger, ParameterFloat, ParameterString`)도 같이 정의합니다.

훈련, 검증 및 테스트 데이터에 대한 S3 경로는 이전 랩처럼 수동으로 지정하는 것이 아니라 체인으로 연결되는 개념이기에, 아래 예시처럼 전처리 스텝 결괏값(`step_process`)의 프로퍼티(`properties`)를 참조하여 지정해야 합니다.
```python
"train": TrainingInput(
    s3_data=step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri
)
```

#### Training Parameter

In [7]:
# training step parameters
training_entry_point = ParameterString(name="TrainingEntryPoint", default_value="train.py")
training_source_dir = ParameterString(name="TrainingSourceDir", default_value="./src")
training_instance_type = ParameterString(name="TrainingInstanceType", default_value="ml.p3.8xlarge")
training_instance_count = ParameterInteger(name="TrainingInstanceCount", default_value=1)

# hyperparameters, which are passed into the training job
n_gpus = ParameterString(name="NumGPUs", default_value="1")
epochs = ParameterString(name="Epochs", default_value="1")
seed = ParameterString(name="Seed", default_value="42")
train_batch_size = ParameterString(name="TrainBatchSize", default_value="1")
eval_batch_size = ParameterString(name="EvalBatchSize", default_value="2")           
learning_rate = ParameterString(name="LearningRate", default_value="5e-5") 

model_id = ParameterString(name="ModelId", default_value=model_id_)
tokenizer_id = ParameterString(name="TokenizerId", default_value=tokenizer_id_)
dataset_name = ParameterString(name="DatasetName", default_value=dataset_name_)

In [8]:
hyperparameters = {
    'n_gpus': n_gpus,                       # number of GPUs per instance
    'epochs': epochs,                       # number of training epochs
    'seed': seed,                           # seed
    'train_batch_size': train_batch_size,   # batch size for training
    'eval_batch_size': eval_batch_size,     # batch size for evaluation
    'warmup_steps': 0,                      # warmup steps
    'learning_rate': learning_rate,         # learning rate used during training
    'tokenizer_id': model_id,               # pre-trained tokenizer
    'model_id': tokenizer_id                # pre-trained model
} 

chkpt_s3_path = f's3://{bucket}/{s3_prefix}/processing/checkpoints'

In [9]:
huggingface_estimator = HuggingFace(
    entry_point="train.py",
    source_dir="./src",
    base_job_name=base_job_prefix + "-training",
    instance_type="ml.p3.8xlarge",
    instance_count=training_instance_count,
    role=role,
    transformers_version=transformers_version,
    pytorch_version=pytorch_version,
    py_version=py_version,
    hyperparameters=hyperparameters,
    sagemaker_session=sagemaker_session,    
    disable_profiler=True,
    debugger_hook_config=False,
    checkpoint_s3_uri=chkpt_s3_path,
    checkpoint_local_path='/opt/ml/checkpoints'
)

step_train = TrainingStep(
    name="TrainModel",
    estimator=huggingface_estimator,
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri
        ),
        "test": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri
        ),
    },
    cache_config=cache_config,
)

### 1.4. Model evaluation Step

훈련된 모델의 성능을 평가하기 위해 추가 `ProcessingStep`을 정의합니다. 평가 결과에 따라 모델이 생성, 등록 및 배포되거나 파이프라인이 중단됩니다.
평가 결과는 `PropertyFile`에 복사되며, 이는 이후 `ConditionStep`에서 사용됩니다.

#### Evaluation Parameter

In [10]:
evaluation_script = ParameterString(name="EvaluationScript", default_value="./src/evaluate.py")
evaluation_instance_type = ParameterString(name="EvaluationInstanceType", default_value="ml.m5.xlarge")
evaluation_instance_count = ParameterInteger(name="EvaluationInstanceCount", default_value=1)

#### Evaluator

In [11]:
!pygmentize ./src/evaluate.py

import subprocess
import sys
import json
import logging
import pathlib
import tarfile
import os

import numpy as np
import pandas as pd


logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

if __name__ == "__main__":
    logger.debug("Starting evaluation.")
    model_path = "/opt/ml/processing/model/model.tar.gz"
    with tarfile.open(model_path) as tar:
        tar.extractall(path="./hf_model")

    logger.debug(os.listdir("./hf_model"))

    with open("./hf_model/evaluation.json") as f:
        eval_result = json.load(f)

    logger.debug(eval_result)
    output_dir = "/opt/ml/processing/evaluation"
    pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)

    evaluation_path = f"{output_dir}/evaluation.json"
    with open(evaluation_path, "w") as f:
        f.write(json.dumps(eval_result))


In [12]:
script_eval = SKLearnProcessor(
    framework_version="1.0-1",
    instance_type="ml.m5.xlarge",
    instance_count=evaluation_instance_count,
    base_job_name=base_job_prefix + "-evaluation",
    sagemaker_session=sagemaker_session,
    role=role,
)

evaluation_report = PropertyFile(
    name="EvaluationReport",
    output_name="evaluation",
    path="evaluation.json",
)

step_eval = ProcessingStep(
    name="EvalLoss",
    processor=script_eval,
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model",
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name="evaluation",
            source="/opt/ml/processing/evaluation",
            destination=f"s3://{bucket}/{s3_prefix}/evaluation_report",
        ),
    ],
    code="./src/evaluate.py",
    property_files=[evaluation_report],
    cache_config=cache_config,
)

### 1.5. Register the model

훈련된 모델은 모델 패키지 그룹(Model Package Group)의 모델 레지스트리(Model Registry)에 등록됩니다. 모델 레지스트리는 SageMaker Pipelines에서 소개된 개념으로, 기존 SageMaker 모델과 다르게 모델 버전 관리가 가능하며 승인 여부를 지정할 수 있습니다. 모델 승인은 `ConditionStep`의 조건을 만족할 때에만 가능하게 할 수 있습니다. (예: 정확도가 95% 이상인 경우에만 모델 배포)

In [13]:
model = HuggingFaceModel(
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    role=role,
    transformers_version=transformers_version,
    pytorch_version=pytorch_version,
    py_version=py_version,
    sagemaker_session=sagemaker_session,
)
model_package_group_name = "LJPModelPackageGroup"
step_register = RegisterModel(
    name="RegisterModel",
    model=model,
    content_types=["application/json"],
    response_types=["application/json"],
    inference_instances=["ml.m5.xlarge", "ml.g4dn.xlarge"],
    transform_instances=["ml.m5.xlarge", "ml.g4dn.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status="Approved",
)

### 1.6. Model Deployment


`LambdaStep`에서 파생된 커스텀 단계 `ModelDeployment`를 생성합니다. LambdaStep에서 정의한 Lambda 함수를 통해 호스팅 리얼타임 엔드포인트를 배포합니다.

In [14]:
!pygmentize utils/deploy_step.py

import time
import json
import boto3
import os
from sagemaker.workflow.step_collections import StepCollection
from sagemaker.workflow._utils import _RegisterModelStep
from sagemaker.lambda_helper import Lambda
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)


class ModelDeployment(StepCollection):
    """custom step to deploy model as SageMaker Endpoint"""

    def __init__(
        self,
        model_name: str,
        registered_model: _RegisterModelStep,
        endpoint_instance_type,
        sagemaker_endpoint_role: str,
        autoscaling_policy: dict = None,
    ):
        self.name = "sagemaker-pipelines-model-deployment"
        self.model_package_arn = registered_model.properties.ModelPackageArn
        self.lambda_role = self.create_lambda_role(self.name)
        #        Use the current time to define unique names for the resources created
        current_time = time.strftime("%m-%d-%H-%M-%S", time.localtime())

 

In [15]:
# custom Helper Step for ModelDeployment
from utils.deploy_step import ModelDeployment

# we will use the iam role from the notebook session for the created endpoint
# this role will be attached to our endpoint and need permissions, e.g. to download assets from s3
sagemaker_endpoint_role=sagemaker.get_execution_role()
model_n_ = "lawcompany/LJP"
model_name = f"{model_n_.split('/')[-1]}-{time.strftime('%Y-%m-%d-%H-%M-%S', time.localtime())}"

step_deployment = ModelDeployment(
    model_name=model_name,
    registered_model=step_register.steps[0],
    endpoint_instance_type="ml.m5.xlarge",
    sagemaker_endpoint_role=sagemaker_endpoint_role,
    autoscaling_policy=None,
)

Using ARN from existing role: sagemaker-pipelines-model-deployment-role


### 1.7. Condition for deployment

`ConditionStep`을 통해 모델 평가 결과를 검사합니다. 정확도가 일정 이상일 때(accuracy > 0.95) 모델 등록 및 배포 파이프라인을 진행합니다.

#### Condition Parameter

In [16]:
threshold_accuracy = ParameterFloat(name="ThresholdAccuracy", default_value=0.95)

#### Condition

In [17]:
cond_gte = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="eval_accuracy",
    ),
    right=threshold_accuracy,
)

step_cond = ConditionStep(
    name="CheckEvalAccuracy",
    conditions=[cond_gte],
    if_steps=[step_register, step_deployment],
    else_steps=[],
)

<br>

## 2. Pipeline definition and execution

---

모든 스텝을 정의하였다면 파이프라인을 정의합니다. 

파이프라인 인스턴스는 이름(`name`), 파라메터(`parameters`), 및 스텝(`steps`)으로 구성됩니다. 
- 파이프라인 이름: (AWS 계정, 리전) 쌍 내에서 고유해야 합니다 
- 파라메터: 스텝 정의에 사용했던 모든 파라메터들을 파이프라인에서 정의해야 합니다. 
- 스텝: 리스트 형태로 이전 스텝들을 정의합니다. 내부적으로 데이터 종속성을 사용하여 각 스텝 간의 관계를 DAG으로 정의하기 때문에 실행 순서대로 나열할 필요는 없습니다.

In [18]:
pipeline = Pipeline(
    name=f"LJP-Pipeline",
    parameters=[
        model_id,
        tokenizer_id,        
        dataset_name,
        processing_instance_type,
        processing_instance_count,
        processing_script,
        training_entry_point,
        training_source_dir,
        training_instance_type,
        training_instance_count,
        evaluation_script,
        evaluation_instance_type,
        evaluation_instance_count,
        threshold_accuracy,
        n_gpus,
        epochs,
        seed,
        eval_batch_size,
        train_batch_size,
        learning_rate,
    ],
    steps=[step_process, step_train, step_eval, step_cond],
    sagemaker_session=sagemaker_session,
)

#### Check the pipeline definition


In [19]:
import json

definition = json.loads(pipeline.definition())
definition

Popping out 'CertifyForMarketplace' from the pipeline definition since it will be overridden in pipeline execution time.


{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'ModelId',
   'Type': 'String',
   'DefaultValue': 'lawcompany/KLAID_LJP_base'},
  {'Name': 'TokenizerId',
   'Type': 'String',
   'DefaultValue': 'lawcompany/KLAID_LJP_base'},
  {'Name': 'DatasetName',
   'Type': 'String',
   'DefaultValue': 'lawcompany/KLAID'},
  {'Name': 'ProcessingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'ProcessingInstanceCount', 'Type': 'Integer', 'DefaultValue': 1},
  {'Name': 'ProcessingScript',
   'Type': 'String',
   'DefaultValue': './src/processing_sklearn.py'},
  {'Name': 'TrainingEntryPoint', 'Type': 'String', 'DefaultValue': 'train.py'},
  {'Name': 'TrainingSourceDir', 'Type': 'String', 'DefaultValue': './src'},
  {'Name': 'TrainingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.p3.8xlarge'},
  {'Name': 'TrainingInstanceCount', 'Type': 'Integer', 'DefaultValue': 1},
  {'Name': 'EvaluationScript',
   'Type': 'String',
   'DefaultValue':

In [20]:
pipeline.upsert(role_arn=role)

Popping out 'CertifyForMarketplace' from the pipeline definition since it will be overridden in pipeline execution time.
Popping out 'CertifyForMarketplace' from the pipeline definition since it will be overridden in pipeline execution time.


{'PipelineArn': 'arn:aws:sagemaker:us-east-1:353411055907:pipeline/ljp-pipeline',
 'ResponseMetadata': {'RequestId': '6f67df4e-05f9-4107-bcf1-989e192df0d0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6f67df4e-05f9-4107-bcf1-989e192df0d0',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '80',
   'date': 'Mon, 22 May 2023 15:14:38 GMT'},
  'RetryAttempts': 0}}

### Run the pipeline

파이프라인을 실행합니다.

In [21]:
execution = pipeline.start()

In [22]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:353411055907:pipeline/ljp-pipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:353411055907:pipeline/ljp-pipeline/execution/jvirkerbvbjv',
 'PipelineExecutionDisplayName': 'execution-1684768479440',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2023, 5, 22, 15, 14, 39, 293000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2023, 5, 22, 15, 14, 39, 293000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': '3f58821d-9801-4d55-9ad8-242070d887c7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3f58821d-9801-4d55-9ad8-242070d887c7',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '389',
   'date': 'Mon, 22 May 2023 15:14:39 GMT'},
  'RetryAttempts': 0}}

In [23]:
execution.wait()

실행된 스텝들을 리스트업합니다.

In [24]:
execution.list_steps()

[{'StepName': 'ModelDeployment',
  'StartTime': datetime.datetime(2023, 5, 22, 15, 32, 43, 657000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2023, 5, 22, 15, 32, 47, 503000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'Lambda': {'Arn': 'arn:aws:lambda:us-east-1:353411055907:function:sagemaker-pipelines-model-deployment-05-22-15-14-34',
    'OutputParameters': [{'Name': 'other_key', 'Value': 'example_value'},
     {'Name': 'body', 'Value': '"Created Endpoint!"'},
     {'Name': 'statusCode', 'Value': '200.0'}]}}},
 {'StepName': 'RegisterModel-RegisterModel',
  'StartTime': datetime.datetime(2023, 5, 22, 15, 32, 42, 145000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2023, 5, 22, 15, 32, 43, 79000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:353411055907:model-package/ljpmodelpackagegroup/2'}}},
 {'StepName': 'CheckEvalAccuracy',
  'StartTime'

<br>

## 3. Getting predictions from the endpoint
---

파이프라인의 모든 단계가 정상적으로 실행되었다면 배포된 엔드포인트를 통해 실시간 추론을 수행할 수 있습니다.

In [27]:
from sagemaker.huggingface import HuggingFacePredictor
endpoint_name = model_name

# check if endpoint is up and running
print(f"https://console.aws.amazon.com/sagemaker/home?region={region}#/endpoints/{endpoint_name}")
hf_predictor = HuggingFacePredictor(endpoint_name,sagemaker_session=sagemaker_session)

https://console.aws.amazon.com/sagemaker/home?region=us-east-1#/endpoints/LJP-2023-05-22-15-14-34


In [28]:
# example request, you always need to define "inputs"
data = {
   "inputs": [
       "피고인은 2018. 8. 9. 23:33경 술을 마신 상태로 경산시 사동에 있는 상호 불상의 식당에서부터 같은 동에 있는 부영5차 앞 삼거리까지 B 스타렉스 승용차를 운전한 다음 승용차 안에서 잠을 자던 중, 차량 운전자가 시동을 걸어 놓고 잠을 자고 있다는 112 신고를 받고 현장에 출동한 경산경찰서 C파출소 소속 경위 D으로부터 피고인의 입에서 술 냄새가 나고 보행이 비틀거리는 등 술에 취한 상태에서 운전하였다고 인정할 만한 상당한 이유가 있어 약 10분 동안 총 3회에 걸쳐 음주측정기에 입김을 불어 넣는 방법으로 음주측정에 응할 것을 요구받고도 정당한 사유 없이 이에 응하지 아니하였다."

   ]
}
hf_predictor.predict(data)

[{'label': '도로교통법 제148조의2 제2항,도로교통법 제44조 제2항', 'score': 0.9968319535255432}]

In [29]:
# example request, you always need to define "inputs"
data = {
   "inputs": [

       "피고인 A은 노동일에 종사 중이다. 피고인은 2017. 2. 2. 20:00경 부산 해운대구 B, C식당 앞 노상에서 술을 마시고 D의 포터차량을 타려다가 주변에 있던 E 일행이 음주운전을 하려 한다고 말을 건 일로 서로 시비가 되었다. 피고인은 피해자 E과 시비를 벌이다. 피해자를 밀치고, 인근 주차장에서 들고 온 약 1.5m 길이의 쇠막대기로 E의 머리와 어깨 부위를 4~5회 가량 내리쳐 피해자를 폭행하였다."

   ]
}
hf_predictor.predict(data)

[{'label': '형법 제260조 제1항', 'score': 0.9990469813346863}]

<br>

## Clean up
---

과금을 방지하기 위해 사용하지 않는 리소스를 삭제합니다. 아래 코드셀은 Lambda 함수와 엔드포인트를 삭제합니다. 

In [ ]:
#sm_client = boto3.client("sagemaker")

# Delete the Lambda function
#step_deployment.func.delete()

# Delete the endpoint
#hf_predictor.delete_endpoint()